### Pydantic Output Parser
언어모델의 출력을 더 구조화된 정보로 변환하는 데 도움이 되는 클래스
단순 텍스트 형태의 응답 대신, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공

핵심 메서드 2개
- `get_format_instructions()` : 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction)을 제공
- `parse()` : 언어모델의 출력을 받아들여 이를 특정 구조로 분석하고 변환. Pydantic와 같은 도구를 사용하여 입력된 문자열을 사전 정의된 스키마에 따라 검증하고 해당 스키마를 따르는 데이터 구조로 변환

In [1]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

llm = ChatOpenAI(temperature=0.1, model_name = 'gpt-4o-mini')


In [4]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [5]:
from itertools import chain
from langchain_core.prompts import PromptTemplate
from langchain_teddynote.messages import stream_response # 실시간 출력

prompt = PromptTemplate.from_template("다음의 이메일 내용 중 중요한 내용을 추출해주세요. \n \n {email_conversation}")

chain = prompt | llm

answer = chain.stream({"email_conversation" : email_conversation})

output = stream_response(answer, return_output = True)

중요 내용 요약:

- 발신자: 김철수 (바이크코퍼레이션 상무)
- 수신자: 이은채 (Teddy International)
- 주제: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
- 요청 사항: ZENESIS 모델의 상세 브로슈어 (기술 사양, 배터리 성능, 디자인 정보)
- 미팅 제안: 다음 주 화요일(1월 15일) 오전 10시, 귀사 사무실에서 만나기 희망.

In [6]:
class EmailSummary(BaseModel):
   person: str = Field(description = "메일을 보낸 사람") 
   email: str = Field(description = "메일을 보낸 사람의 이메일 주소")  
   subject: str = Field(description = "메일 제목")
   summary: str = Field(description = "메일 본문을 요약한 텍스트")
   date: str = Field(description = "메일 본문에 언급된 미팅 날짜와 시간")
   

# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [7]:
prompt = PromptTemplate.from_template(
   """
   Question: {question},
   Email Conversation: {email_conversation},
   Format: {format}
   """
)

prompt = prompt.partial(format=parser.get_format_instructions())

chain = prompt | llm

response = chain.stream(
   {
      "email_conversation": email_conversation,
      "question": "이메일 내용중 주요 내용을 추출해 주세요.",
   }
)

output = stream_response(response, return_output=True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 유통 협력 가능성을 논의하기 위해 ZENESIS 모델의 상세 브로슈어 요청과 미팅 제안을 함.",
  "date": "2024-01-15T10:00:00"
}
```

In [8]:
structured_output = parser.parse(output)  
print(structured_output)

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='김철수 상무가 이은채 대리님에게 바이크코퍼레이션의 자전거 유통 협력 가능성을 논의하기 위해 ZENESIS 모델의 상세 브로슈어 요청과 미팅 제안을 함.' date='2024-01-15T10:00:00'


### JSON Output Parser

- 사용자가 원하는 JSON 스키마를 지정할 수 있게 해줌
- 그 스키마에 맞게 LLM에서 데이터를 조회하여 결과를 도출해줌
- 데이터를 정확하고 효율적으로 처리하여 원하는 형태의 JSON을 생성하기 위해서는 모델의 용량이 충분해야함

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

